In [1]:
import time
from torch.utils.tensorboard import SummaryWriter
import os
import surpport.mySQL as mySQL
import surpport.myfunction as MF
from surpport.Args import Args
import torch.nn.functional as F
import torch
import numpy as np
from surpport.nnstructure import Classifier7

Using backend: pytorch


In [5]:
def recorder_build(arg):
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    return recorder


def train(arg, model, recorder, opt, tr_dataloader, ts_dataloader, writer):
    # train
    for epoch in range(1, arg.epoch_num+1):
        # print(epoch)
        recorder[str(epoch)+'-th'] = dict()
        rd = recorder[str(epoch)+'-th']
        # 由于图经过拼合，所以需要多一个dataloader的过程
        # 前两个是list
        tr_loss, tr_acc = MF.train(
            epoch,
            model, opt, tr_dataloader,
            arg,
            writer
        )
        mySQL.rcd_log(tr_loss, tr_acc, writer, rd, epoch, 'train')

        el_loss, el_acc, logits, labels = MF.evaluate(
            epoch,
            model, opt, ts_dataloader,
            arg,
            writer
        )

        mySQL.rcd_log(el_loss, el_acc, writer, rd, epoch, 'test')
        mySQL.rcd_result(logits, labels, rd)

        val_acc = np.mean(el_acc, axis=0)
        mySQL.save_best(epoch, model, val_acc, arg, opt)

    mySQL.save_final(epoch, model, val_acc, arg, opt)

    mySQL.save_recorder(recorder, arg, 'flow')
    print('best acc: %.4f' % (arg.best_acc))
    return None


## cross 超参数实验

In [3]:
heads = [3, 6, 12, 24]
# alphas = [5, 7, 9, 11]
alphas = [9]*4 # 训练没保存参数，得重新来
pairs = []
for h in heads:
    pairs.append((h, 9))
pairs

[(3, 9), (6, 9), (12, 9), (24, 9)]

In [4]:
## arg 属性调整
from surpport.Args import Args
def rec_arg(arg: Args, p):
    arg.pip_num = p[0]
    arg.display_freq = 80
    arg.exception_acc = 0.05
    arg.epoch_num = 120
    return arg

In [ ]:
from surpport.cross_Dprocess import c_dataload

count = 1

for p in pairs[1:]:
    arg = rec_arg(Args(), p)
    model = Classifier7(arg) # 都不接受参数，所以没问题
    model = model.cuda()
    arg.d_prepare('Cross', 2)
    # ! 每次运行都改
    count += 1
    arg.m_info(m_name='m7', m_task='220920_pre_exp', num=count,)
    base_rcd = mySQL.gen_base_rcd(arg)
    recorder = {'base': base_rcd}
    writer = SummaryWriter(arg.tar_path+'\\Journal')

    opt_arg = {'params': model.parameters()}
    #    'lr': 6e-5, 'eps': 1e-8, 'weight_decay': 0.1}
    opt = torch.optim.Adam(**opt_arg)

    arg, tr_dataloader, ts_dataloader = c_dataload(arg, model, opt, p[1])

    train(arg, model, recorder, opt, tr_dataloader, ts_dataloader, writer)
    writer.close()